In [1]:
import re
from asyncio import Semaphore
from typing import Literal

import fuzzysearch
import pandas as pd
from litellm import acompletion
from openpyxl import Workbook
from openpyxl.cell.rich_text import CellRichText, TextBlock
from openpyxl.cell.text import InlineFont
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.worksheet import Worksheet
from tqdm.asyncio import tqdm
from unidecode import unidecode

tqdm.pandas()

## Preprocess and filter the claims extracted during the ECF Hackaton


In [2]:
df = pd.read_parquet("../../data/raw/4_channels_predictions_09_2023_09_2024.parquet")
df

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claims
id,,,,,,,,,,,
1dcd4b454f8bac42440259ce26a1a2192051186bb5728be489dc654a9a967d1d,2023-09-01 06:26:00,<unk> <unk> <unk> <unk> aerosmith en janvier m...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""eaux"", ""timestamp"": 169354246806...",1,383,[{'analysis': 'Cette allégation nécessite une ...
0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc93f701fd56bd8ea66f,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,[{'analysis': 'L'allégation semble être fondée...
b6d54aefb250671e7754a688411ce9e68badcad88a665be3de78996d13b74fd2,2023-09-01 07:38:00,mais titeuf ne vieillit pas le monde change ti...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_constat""]","[{""keyword"": ""bilan carbone"", ""timestamp"": 169...",1,606,[{'analysis': 'Il est pertinent de vérifier si...
23c2d3b292d9ab0fb0d0b2b8c34f3b88708c79ffab2d6659accf565ba61f48ae,2023-09-01 08:44:00,dû travailler très vite le journal arrive à no...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""eaux"", ""timestamp"": 169355074908...",3,515,[{'analysis': 'Cette allégation est problémati...
71df0ce2b34afa23391d8e31d35ccd213ae2a881b7ce412813a06a60a2e47d3c,2023-09-01 08:46:00,pas avoir lieu ni même européens existait ni l...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""\u00e9cologiste"", ""timestamp"": 1...",2,536,[{'analysis': 'L'allégation concernant la chut...
...,...,...,...,...,...,...,...,...,...,...,...
7e43ce79a3c9304277dde3a06b433d7cf08a9b40ee8904be3fdedea1cd608d36,2024-08-31 13:08:00,lui aussi par les moustiques touche une trenta...,tf1,False,Information - Journal,JT 13h,"[""biodiversite_consequences"", ""biodiversite_co...","[{""keyword"": ""renaturer"", ""timestamp"": 1725102...",4,474,[{'analysis': 'Cette allégation est fondée sur...
4b5a990a668084eb50da9029b16f7f76ba44ffc89412858aff2789f79efabcec,2024-08-31 13:10:00,des griffes arracher la solution se trouve peu...,tf1,False,Information - Journal,JT 13h,"[""biodiversite_consequences"", ""attenuation_cli...","[{""keyword"": ""agriculture"", ""timestamp"": 17251...",3,471,[{'analysis': 'L'allégation qu'une plante a de...
d3ef252aaac62bdcd506e162d5454dfa346f9c75dbc74efb675e408409ae7386,2024-08-31 19:58:00,albret <unk> madame monsieur bonsoir bienvenue...,tf1,False,Information - Journal,JT 20h + météo,"[""changement_climatique_consequences""]","[{""keyword"": ""al\u00e9as climatiques"", ""timest...",1,419,[{'analysis': 'L'allégation sur la qualité de ...


In [3]:
# Keep only the "high-level" claims
df["claims"] = df["claims"].apply(
    lambda claims: [
        claim for claim in claims if claim["disinformation_score"] == "high"
    ]
)
df = df[df["claims"].str.len() > 0].copy()
len(df)

1494

In [4]:
# Less than 10% of claims have more than one claim per extract, let's keep just one for simplicity
# We keep the first one when there are several claims
display(df["claims"].str.len().quantile([0.9, 0.95, 0.99, 1]))
df["claims"] = df["claims"].str[0]

0.90    1.0
0.95    2.0
0.99    2.0
1.00    3.0
Name: claims, dtype: float64

In [5]:
# Now that we have one claim per extract we can flatten the list of claims into columns
claims = pd.json_normalize(df["claims"])
claims.index = df.index
df = pd.concat([df, claims], axis=1).drop(columns="claims")
df

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,analysis,claim,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker
id,,,,,,,,,,,,,,,,,,,
f17cb36a314156417616ef1ebb9232bbf203a1ecdc983c796a1a8ddba78393da,2023-09-04 05:56:00,série qu'on peut redire par fonction juré déjà...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169379984302...",2,559,Cette allégation semble associée à une interpr...,le giec est transhumaniste,Dans un passage en parlant des personnages du ...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative
e57e39657004072e5f998d19e6a8a952ee8f35923b9072f556031374e9150d2e,2023-09-04 07:50:00,salle d'attente il y a toutes les classes rage...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""giec"", ""timestamp"": 169380664205...",2,554,Cette allégation semble utiliser le terme 'tra...,"La mention du GIEC comme un ""transhumaniste"".",Dans un extrait de programme où différents per...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative
5b7207073227562ad5fd7b252e290d0a2079f3cbbb2e5cd6d81624fc22cc2a1e,2023-09-06 07:22:00,en france on peut quand même s'appuyer sur ce ...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_consequences_indirecte...","[{""keyword"": ""incendies"", ""timestamp"": 1693977...",2,594,Cette allégation semble hautement exagérée et ...,Il y a eu beaucoup de morts tués par des pharm...,Lors d'une discussion sur le rôle des pharmaci...,N/A,narrative,high,anti-écologie,j'ai l'impression qu'il y a eu beaucoup de mor...,narrative
53b6b045672a2327c58c72a38a3ddb603587ad43421f8c84f3a70cbf401a78d5,2023-09-06 08:38:00,je suis même allé plus loin je vais essayer de...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_consequences"", ""ressou...","[{""keyword"": ""\u00e9coresponsable"", ""timestamp...",2,614,"Cette allégation frôle la théorie du complot, ...",Le réchauffement climatique est causé par une ...,Lors d'une discussion sur les enjeux environne...,N/A,consensus,high,anti-écologie,Le réchauffement climatique est causé par une ...,narrative
6c3bb9299d31332c644b54832399976117d28776c9e22ca642a957835ade828d,2023-09-07 08:56:00,pour faire réussir nos enfants dans quel monde...,europe1,True,Information - Magazine,L'heure des pros,"[""changement_climatique_constat""]","[{""keyword"": ""climatique"", ""timestamp"": 169406...",1,563,Si le changement climatique est parfois utilis...,Le changement climatique est traité comme une ...,Le commentaire sur une polémique entre personn...,N/A,consensus,high,anti-écologie,...c'était hier soir dans cette à vous sur fra...,narrative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9441e27a4c144b4ceef27cb8bd44ae5b7ae0fd545fde2ba8183bfa8e7dec66fc,2024-08-06 09:04:00,chute de séracs sont communs commune à toutes ...,tf1,False,Information - Magazine,Bonjour ! La Matinale,"[""changement_climatique_constat_indirectes"", ""...","[{""keyword"": ""climatique"", ""timestamp"": 172292...",1,400,"Cette affirmation semble contestable, car de n...",Le risque de chutes de séracs n'est pas partic...,"Le chercheur indique que, pour l'instant, le r...",N/A,consensus,high,anti-écologie,"Pour l'instant, le risque de chutes de séracs ...",narrative
deced996c604c36119a2b873d7ce1b0db917c20787118640a6590d77bd914a3a,2024-08-15 08:36:00,voyageurs à reporter leurs déplacements et à n...,tf1,False,Information - Magazine,Bonjour ! La Matinale,"[""changement_climatique_constat"", ""ressources_...","[{""keyword"": ""incendies"", ""timestamp"": 1723703...",3,515,"Cette allégation est assez sérieuse, car elle ...",les forêts ont absorbé deux fois moins de CO2 ...,Le commenta

In [6]:
# Keeping only the "anti-écologie" claims
display(df["pro_anti"].value_counts())
df = df[df["pro_anti"] == "anti-écologie"].copy()
len(df)

pro_anti
anti-écologie    1377
pro-écologie      117
Name: count, dtype: int64

1377

In [ ]:
# Try to retrieve the start & end indices of the identified quotes for each claim
# Since the LLM doesn't like typos and so on, it will rarely perfectly quote the original text
# Therefore we need to do approximate matching
# This matching code can be improved, it's still a bit wobbly


def approximate_match(text: str, quote: str) -> tuple[int, int] | None:
    # Preprocess text to remove casing and accents
    text = unidecode(text.lower())
    quote = unidecode(quote.lower())

    match = fuzzysearch.find_near_matches(
        quote, text, max_l_dist=min(int(0.2 * len(quote)), 20)
    )

    if match:
        return match[0].start, match[0].end


# If we can't retrieve the quote from the original text, we simply ditch this identified claim for now
df["match"] = df.progress_apply(
    lambda x: approximate_match(x["text"], x["quote"]), axis=1
)
display((~df["match"].isna()).value_counts())

df = df[~df["match"].isna()].copy()

df["quote_start"] = df["match"].str[0]
df["quote_end"] = df["match"].str[1]
df = df.drop(columns="match")
len(df)

100%|██████████| 1377/1377 [00:24<00:00, 56.31it/s]


match
True     1065
False     312
Name: count, dtype: int64

1065

## Add the CARDS classification (0 to 7)


In [ ]:
# Limit concurrency
semaphore = Semaphore(100)


async def classify_cards(
    text: str, quote: str
) -> Literal[
    "0_accepted",
    "1_its_not_happening",
    "2_humans_not_the_cause",
    "3_impacts_not_bad",
    "4_solutions_harmful_unnecessary",
    "5_science_uncertain",
    "6_proponents_biased",
    "7_fossil_fuels_needed",
]:
    messages = [
        {
            "role": "system",
            "content": """Tu es un expert en désinformation sur les questions environnementales, spécialiste en science climatique et bien informé sur le GIEC. L'utilisateur va te fournir un extrait de deux minutes d'une transcription d'un programme télévisé ou radiophonique. Dans cet extrait, nous avons repéré un passage qui pourrait contenir des propos climatosceptiques nécessitant un fack-checking. Ta mission est de classer ce passage dans la catégorie appropriée. Note que certains passages peuvent avoir été identifiés par erreur. Concentre-toi uniquement sur les passages clairement climatosceptiques. Si tu as un doute, attribue-les à la catégorie 0_accepted.

# Catégories prédéfinies

- 0_accepted: Le passage n'a pas de contenu climatosceptique évident et est accepté tel quel.
- 1_its_not_happening: Négation du réchauffement climatique ou allégations selon lesquelles les signes de changement climatique ne seraient pas avérés, tels que la fonte des glaces ou l'élévation du niveau de la mer.
- 2_humans_not_the_cause: Refus de reconnaître que les gaz à effet de serre produits par l'activité humaine sont la cause du changement climatique, attribuant plutôt cela à des variations naturelles ou autres phénomènes.
- 3_impacts_not_bad: Minimisation des impacts du changement climatique, suggérant qu'ils pourraient être bénins ou même bénéfiques.
- 4_solutions_harmful_unnecessary: Critique des solutions climatiques, affirmant qu'elles sont nuisibles, inutiles, ou inefficaces.
- 5_science_uncertain: Contestation de la validité de la science climatique, affirmant qu'elle est incertaine, biaisée ou peu fiable.
- 6_proponents_biased: Accusations de partialité ou de motivations politiques parmi les scientifiques et les défenseurs de l'action climatique, allant jusqu'à suggérer un alarmisme ou une conspiration.
- 7_fossil_fuels_needed: Arguments en faveur de l'utilisation continue des combustibles fossiles pour le développement économique et le maintien du niveau de vie.

# Exemples

## Exemple 0

<extrait>
[...]
</extrait>
<passage climatosceptique>
les scientifiques observent une hausse des températures mondiales, entraînant des changements climatiques visibles ils soulignent la nécessité d'agir pour en limiter les impacts
</passage climatosceptique>
<catégorie>0_accepted</catégorie>

## Exemple 1

<extrait>
[...]
</extrait>
<passage climatosceptique>
la fréquence des discussions sur la crise climatique semble augmenter mais certains experts proposent que les avertissements sur le climat pourraient être exagérés faisant souvent valoir que les fluctuations climatiques sont tout à fait naturelles et que les vérités scientifiques sont parfois altérées pour servir des intérêts politiques
</passage climatosceptique>
<catégorie>1_its_not_happening</catégorie>

## Exemple 2

<extrait>
[...]
</extrait>
<passage climatosceptique>
il est contesté que la contribution humaine aux niveaux de CO2 soit réellement significative certains experts soutiennent que les océans jouent un rôle bien plus important dans l'émission de ce gaz
</passage climatosceptique>
<catégorie>2_humans_not_the_cause</catégorie>

## Exemple 3

<extrait>
[...]
</extrait>
<passage climatosceptique>
il est important de noter que beaucoup de régions touchées par des événements climatiques extrêmes retrouvent leur équilibre naturel et peuvent même bénéficier de ces changements car plusieurs études montrent que certaines espèces et écosystèmes s'adaptent efficacement aux variations climatiques
</passage climatosceptique>
<catégorie>3_impacts_not_bad</catégorie>

## Exemple 4

<extrait>
[...]
</extrait>
<passage climatosceptique>
de nombreux experts remettent en question l'efficacité des initiatives visant à réduire la pollution, arguant qu'elles ne font qu'ajouter des coûts économiques sans réel impact positif sur l'environnement
</passage climatosceptique>
<catégorie>4_solutions_harmful_unnecessary</catégorie>

## Exemple 5

<extrait>
[...]
</extrait>
<passage climatosceptique>
En réfléchissant à la situation on peut voir un parallèle avec le débat climatique de nombreux sceptiques pensent que le discours autour du climat est devenu une sorte de dogme où la critique est difficilement tolérée
</passage climatosceptique>
<catégorie>5_science_uncertain</catégorie>

## Exemple 6

<extrait>
[...]
</extrait>
<passage climatosceptique>
Certains médias et politiciens utilisent la peur du changement climatique pour avancer des agendas politiques partisans, affirmant que les scientifiques amplifient les données pour obtenir plus de subventions.
</passage climatosceptique>
<catégorie>6_proponents_biased</catégorie>

## Exemple 7

<extrait>
[...]
</extrait>
<passage climatosceptique>
L'économie mondiale repose encore largement sur les combustibles fossiles, et leur abandon mettrait en péril notre développement économique et notre sécurité énergétique, car les énergies renouvelables ne sont pas encore prêtes à prendre le relais.
</passage climatosceptique>
<catégorie>7_fossil_fuels_needed</catégorie>""",
        },
        {
            "role": "user",
            "content": f"""<extrait>
{text}
</extrait>
<passage climatosceptique>
{quote}
</passage climatosceptique>""",
        },
    ]
    async with semaphore:
        response = await acompletion(
            "gpt-4o-mini",
            messages=messages,
            stop="</",
            temperature=0,
        )
    cards_class = response.choices[0].message.content.split("<catégorie>")[1]
    return cards_class

In [9]:
# Run the CARDS classification inference for all the claims
df["cards"] = await tqdm.gather(
    *[
        classify_cards(text, quote)
        for text, quote in df[["text", "quote"]].itertuples(index=False)
    ]
)

100%|██████████| 1065/1065 [00:37<00:00, 28.64it/s]


In [10]:
# Save results to avoid recomputing them
df.to_csv("../../data/raw/4_channels_predictions_with_cards_09_2023_09_2024.csv")

In [ ]:
# Reload the results (on notebook restart)
df = pd.read_csv(
    "../../data/raw/4_channels_predictions_with_cards_09_2023_09_2024.csv", index_col=0
)
df

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,...,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker,quote_start,quote_end,cards
id,,,,,,,,,,,,,,,,,,,,,
f17cb36a314156417616ef1ebb9232bbf203a1ecdc983c796a1a8ddba78393da,2023-09-04 05:56:00,série qu'on peut redire par fonction juré déjà...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169379984302...",2,559,...,Dans un passage en parlant des personnages du ...,NaN,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative,1533,1646,0_accepted
e57e39657004072e5f998d19e6a8a952ee8f35923b9072f556031374e9150d2e,2023-09-04 07:50:00,salle d'attente il y a toutes les classes rage...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""giec"", ""timestamp"": 169380664205...",2,554,...,Dans un extrait de programme où différents per...,NaN,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative,882,957,0_accepted
5b7207073227562ad5fd7b252e290d0a2079f3cbbb2e5cd6d81624fc22cc2a1e,2023-09-06 07:22:00,en france on peut quand même s'appuyer sur ce ...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_consequences_indirecte...","[{""keyword"": ""incendies"", ""timestamp"": 1693977...",2,594,...,Lors d'une discussion sur le rôle des pharmaci...,NaN,narrative,high,anti-écologie,j'ai l'impression qu'il y a eu beaucoup de mor...,narrative,264,369,0_accepted
6c3bb9299d31332c644b54832399976117d28776c9e22ca642a957835ade828d,2023-09-07 08:56:00,pour faire réussir nos enfants dans quel monde...,europe1,True,Information - Magazine,L'heure des pros,"[""changement_climatique_constat""]","[{""keyword"": ""climatique"", ""timestamp"": 169406...",1,563,...,Le commentaire sur une polémique entre personn...,NaN,consensus,high,anti-écologie,...c'était hier soir dans cette à vous sur fra...,narrative,197,308,0_accepted
52061c3902c0257c7bfae7086ae50ea3998fea4204bcd6628588e41d71340dfc,2023-09-08 19:34:00,c'est non seulement que les utilisent pas mais...,europe1,True,Information - Magazine,Soir,"[""biodiversite_concepts_generaux"", ""biodiversi...","[{""keyword"": ""d\u00e9carboner"", ""timestamp"": 1...",6,449,...,L'intervenant mentionne une perte de confiance...,NaN,consensus,high,anti-écologie,les climatologues ne croient plus leurs termes...,narrative,808,862,5_science_uncertain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f58921002bc9d8158f722c26c510212340b1a1666fc2e58b75ef25954392168a,2024-07-24 09:08:00,de juin dans cette région le préfet a donc déc...,tf1,False,Information - Magazine,Bonjour ! La Matinale,"[""adaptation_climatique_solutions"", ""ressource...","[{""keyword"": ""agriculteur"", ""timestamp"": 17218...",1,505,...,Face à la menace que représentent les loups po...,NaN,consensus,high,anti-écologie,nous on partirait plutôt sur de la capture eff...,facts,1059,1135,0_accepted
f875741067fda8011d4aa9ad4c543e52227e31b525fed1953dbe1d910b4c71b3,2024-07-29 13:14:00,sur place voilà ce qu'ils restent par exemple ...,tf1,False,Information - Journal,JT 13h,"[""changement_climatique_constat_indirectes"", ""...","[{""keyword"": ""feux"", ""timestamp"": 172225164506...",1,446,...,Le journaliste fait état des enquêtes en cours...,NaN,consensus,high,anti-écologie,d'après les autorités le feu serait d'origine ...,narrative,1151,1207,0_accepted
cdfed233896d49971897df448465dd43b79ae871b33adb5a3a5d464755e8a7cb,2024-08-05 09:16:00,les insectes non on pense que c'est la chaleur...,tf1,False,Information - Magazine,Bonjour ! La Matinale,"[""changement_climatique_constat_indirectes"", ""...","[{""keyword"": ""g\u00e9othermie"", ""timestamp"": 1...",2,479,...,"Il est mentionné que la climatisation, qui est...",NaN,consensus,hi

In [13]:
# We can get rid of 60% of the claims that are likely false positives
display(df["cards"].value_counts().sort_index())
display(
    str(round((1 - (df["cards"] == "0_accepted").sum() / len(df)) * 100))
    + "% of the detected claims from the first LLM prompt are likely false positives "
)
df = df[df["cards"] != "0_accepted"].copy()
len(df)

cards
0_accepted                         501
1_its_not_happening                165
2_humans_not_the_cause              42
3_impacts_not_bad                   70
4_solutions_harmful_unnecessary    112
5_science_uncertain                 72
6_proponents_biased                 86
7_fossil_fuels_needed               17
Name: count, dtype: int64

'53% of the detected claims from the first LLM prompt are likely false positives '

564

## Improve the Mediatree text readability for human review


In [ ]:
# Limit concurrency
semaphore = Semaphore(100)


async def improve_mediatree_readability(text: str, quote: str) -> str:
    messages = [
        {
            "role": "system",
            "content": """L'utilisateur va fournir un extrait de 2 minutes d'une émission de télévision ou de radio ainsi qu'une citation extraite du texte. La transcription fournie peut manquer de ponctuation et être de qualité médiocre, avec un vocabulaire incorrect ou un découpage maladroit. Elle sera phonétique. Lorsque le texte semble incohérent, reformule-le en tenant compte de la phonétique, afin d'obtenir un texte final en français correct.

## Exemple

<extrait>
les les energie renouvellable doive etre remi euh en question en europ sané col debuts
</extrait>
<citation à mettre en gras>sané col debuts</citation à mettre en gras>"
<extrait corrigé>
Les énergies renouvelables doivent être remises en question en Europe, **et ça n'est que le début !**
</extrait corrigé>

## Notes

Le format de sortie doit être en Markdown, avec la citation extraite du texte **<citation>** mise en gras dans l'extrait corrigé.
""",
        },
        {
            "role": "user",
            "content": f"""<extrait>
{text}
</extrait>
<citation à mettre en gras>
{quote}
</citation à mettre en gras>""",
        },
    ]
    async with semaphore:
        response = await acompletion(
            "gpt-4o-mini",
            messages=messages,
            stop="</",
            temperature=0,
        )
    fixed_text = (
        response.choices[0].message.content.split("<extrait corrigé>")[1].strip()
    )
    return fixed_text

In [38]:
# Run the text improvement inference for all the claims
df["improved_text_with_quote"] = await tqdm.gather(
    *[
        improve_mediatree_readability(text, quote)
        for text, quote in df[["text", "quote"]].itertuples(index=False)
    ]
)

100%|██████████| 564/564 [00:49<00:00, 11.32it/s]


In [44]:
# Save results to avoid recomputing
df.to_csv(
    "../../data/raw/4_channels_predictions_with_cards_rewritten_09_2023_09_2024.csv"
)

In [80]:
# Reload the results (on notebook restart)
df = pd.read_csv(
    "../../data/raw/4_channels_predictions_with_cards_rewritten_09_2023_09_2024.csv",
    index_col=0,
)
df

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,...,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker,quote_start,quote_end,cards,improved_text_with_quote
id,,,,,,,,,,,,,,,,,,,,,
52061c3902c0257c7bfae7086ae50ea3998fea4204bcd6628588e41d71340dfc,2023-09-08 19:34:00,c'est non seulement que les utilisent pas mais...,europe1,True,Information - Magazine,Soir,"[""biodiversite_concepts_generaux"", ""biodiversi...","[{""keyword"": ""d\u00e9carboner"", ""timestamp"": 1...",6,449,...,NaN,consensus,high,anti-écologie,les climatologues ne croient plus leurs termes...,narrative,808,862,5_science_uncertain,Ce n'est pas seulement qu'ils ne les utilisent...
34a41bf34b35ee91fc147601fb8c21a366a2f568b9060bbb629698dfd9319801,2023-09-15 19:30:00,jusqu'au trente septembre détaille sofia au qu...,europe1,True,Information - Magazine,Soir,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""terre"", ""timestamp"": 16947991040...",2,533,...,NaN,consensus,high,anti-écologie,Il y a beaucoup de soldats poneys menteurs à t...,narrative,1800,1891,5_science_uncertain,"Jusqu'au trente septembre, détaille Sofia au q..."
0fb8db32982baea27fa4a92220e76331e703d61852c0e8d95550ef6853ffd842,2023-09-20 07:50:00,tente d'échapper à une tempête de pluie d'acid...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""biodiversi...","[{""keyword"": ""acide"", ""timestamp"": 16951890010...",3,545,...,NaN,consensus,high,anti-écologie,tempête de pluie d'acide mortelle selon l'inrs...,other,23,132,1_its_not_happening,On tente d'échapper à une **tempête de pluie d...
4792f93c6614b1e7ef39e301cc6c1d0f4d3d18b9421cc3a9b18aa5c7581c9e02,2023-10-02 07:54:00,de français n'arrive pas à se loger c'est offi...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""eaux"", ""timestamp"": 169622615203...",6,573,...,NaN,narrative,high,anti-écologie,Ça va de l'interdiction des avions d'affaires ...,consensus,806,947,3_impacts_not_bad,"Les Français n'arrivent pas à se loger, c'est ..."
74c05fdbca4aeffb643abf0de486b57f8299051d4ef71b58b532791a33aba423,2023-10-05 19:54:00,climatique le réchauffement collecte de grenob...,europe1,True,Information - Magazine,Soir,"[""changement_climatique_constat_indirectes"", ""...","[{""keyword"": ""terre"", ""timestamp"": 16965285020...",1,499,...,Elle contredit les impacts largement documenté...,consensus,high,anti-écologie,on ne peut pas faire le lien directement avec ...,consensus,1316,1378,2_humans_not_the_cause,Le réchauffement climatique a été collecté à G...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22d30e47c43d8defe1bdab51e6e4c6374ba982d76de8896c2abb711950121329,2024-04-23 20:36:00,plus chaud de ces dix dernières années le temp...,tf1,False,Information - Journal,JT 20h + météo,"[""changement_climatique_consequences_indirecte...","[{""keyword"": ""planter des arbres"", ""timestamp""...",5,367,...,NaN,consensus,high,anti-écologie,Il n'y a plus ni eau ni printemps.,consensus,114,148,1_its_not_happening,"Plus chaud de ces dix dernières années, le tem..."
58feacdc4a4d31023a87bce158ee902d37076d9f5562f3fe41f768ba815b7b6a,2024-04-28 20:40:00,hui ce fameux pacte d'immigration qui a été vo...,tf1,False,Information - Journal,JT 20h + météo,"[""ressources_indirectes"", ""ressources_solution...","[{""keyword"": ""agriculteur"", ""timestamp"": 17143...",2,614,...,NaN,narrative,high,anti-écologie,ce tsunami de normes de contraintes de taxe qu...,narrative,1492,1625,4_solutions_harmful_unnecessary,"Aujourd'hui, ce fameux pacte d'immigration qui..."
6d39a0d45c359fd2183c0f1f001b76c4b60d283240e5c34308b71c22c78217ec,2024-06-16 18:04:00,avec l'aide de nos fournisseurs et si il s'avè...,tf1,False,Information - Magazine,Sept à huit Life,"[""ressources_solutions_indirectes"", ""biodivers...","[{""keyword"": "

In [81]:
# Remove the few claims that came up with multiple or no quotes in bold
display((df["improved_text_with_quote"].str.count(r"\*\*") != 2).sum())
df = df[df["improved_text_with_quote"].str.count(r"\*\*") == 2].copy()

np.int64(19)

## Create Excel file for human review


In [92]:
# Keep only interesting columns
review_df = df.reset_index()
review_df = review_df[
    [
        "id",
        "channel_name",
        "start",
        "improved_text_with_quote",
        "quote",
        "claim",
        "analysis",
        "context",
        "cards",
    ]
].rename(columns={"improved_text_with_quote": "text"})

# Sort extracts in case there are follow-up extracts
review_df = review_df.sort_values(by=["channel_name", "start"])

review_df

,id,channel_name,start,text,quote,claim,analysis,context,cards
0,52061c3902c0257c7bfae7086ae50ea3998fea4204bcd6...,europe1,2023-09-08 19:34:00,Ce n'est pas seulement qu'ils ne les utilisent...,les climatologues ne croient plus leurs termes...,Les climatologues ne croient plus leurs termes...,Cette affirmation pourrait créer un doute sur ...,L'intervenant mentionne une perte de confiance...,5_science_uncertain
1,34a41bf34b35ee91fc147601fb8c21a366a2f568b9060b...,europe1,2023-09-15 19:30:00,"Jusqu'au trente septembre, détaille Sofia au q...",Il y a beaucoup de soldats poneys menteurs à t...,Il y a beaucoup de soldats poneys menteurs à t...,Cette allégation semble suggérer que le réchau...,"Lors d'une conférence de presse au Vietnam, le...",5_science_uncertain
2,0fb8db32982baea27fa4a92220e76331e703d61852c0e8...,europe1,2023-09-20 07:50:00,On tente d'échapper à une **tempête de pluie d...,tempête de pluie d'acide mortelle selon l'inrs...,tempête de pluie d'acide mortelle enregistrée ...,L'allégation mentionne une tempête de pluie d'...,Dans une discussion sur un film catastrophe éc...,1_its_not_happening
3,4792f93c6614b1e7ef39e301cc6c1d0f4d3d18b9421cc3...,europe1,2023-10-02 07:54:00,"Les Français n'arrivent pas à se loger, c'est ...",Ça va de l'interdiction des avions d'affaires ...,Diminuer le nombre de vaches et contrôler les ...,Cette affirmation présente des mesures qui n'o...,L'interlocuteur mentionne plusieurs mesures ex...,3_impacts_not_bad
4,74c05fdbca4aeffb643abf0de486b57f8299051d4ef71b...,europe1,2023-10-05 19:54:00,Le réchauffement climatique a été collecté à G...,on ne peut pas faire le lien directement avec ...,On ne peut pas faire le lien directement avec ...,Cette assertion minimise le consensus scientif...,Luc Moreau énonce lui-même qu'il est difficile...,2_humans_not_the_cause
...,...,...,...,...,...,...,...,...,...
540,22d30e47c43d8defe1bdab51e6e4c6374ba982d76de889...,tf1,2024-04-23 20:36:00,"Plus chaud de ces dix dernières années, le tem...",Il n'y a plus ni eau ni printemps.,Il n'y a plus ni eau ni printemps.,Cette affirmation semble exagérée et pourrait ...,L'orateur évoque un changement climatique qui ...,1_its_not_happening
541,58feacdc4a4d31023a87bce158ee902d37076d9f5562f3...,tf1,2024-04-28 20:40:00,"Aujourd'hui, ce fameux pacte d'immigration qui...",ce tsunami de normes de contraintes de taxe qu...,Le pacte vert européen organise la décroissance.,Cette affirmation attribue une intention de dé...,Lors d'un discours critique à l'égard des poli...,4_solutions_harmful_unnecessary
542,6d39a0d45c359fd2183c0f1f001b76c4b60d283240e5c3...,tf1,2024-06-16 18:04:00,"Avec l'aide de nos fournisseurs, si il s'avère...",l'entreprise n'a pas voulu nous dire combien d...,L'huile de palme serait aujourd'hui une soluti...,Cette allégation semble minimiser les impacts ...,Dans un débat sur les pratiques de certains fo...,4_solutions_harmful_unnecessary
543,0718ef4df3559735ad33436f7e1e39802a71d923cbc0b9...,tf1,2024-07-21 17:14:00,"En France, plusieurs millions de maisons sont ...",le dérèglement climatique phénomène naturel.,Le dérèglement climatique est un phénomène nat...,Cette affirmation contredit le consensus scien...,Une affirmation selon laquelle les changements...,2_humans_not_the_cause


In [ ]:
# Pandas dataframe to Excel
writer = pd.ExcelWriter(
    "../../data/raw/4_channels_review_09_2023_09_2024.xlsx", engine="openpyxl"
)
review_df.to_excel(writer, sheet_name="Claims review", index=False)

workbook: Workbook = writer.book
worksheet: Worksheet = writer.sheets["Claims review"]

# Styling: Wrap text + vertical align top
for rows in worksheet.iter_rows():
    for cell in rows:
        cell.alignment = Alignment(wrapText=True, vertical="top")

# Replace the "text" column with a bolded version
for i, text in enumerate(review_df["text"], start=2):
    # Find quote start and end using the **bold** markers
    bold_search = re.search(r"\*\*(.*?)\*\*", text, flags=re.M)
    quote_start = bold_search.start()
    quote_end = bold_search.end()

    # "text" will be in the "D" column
    column_letter = get_column_letter(review_df.columns.tolist().index("text") + 1)
    row_number = str(i)
    worksheet[f"{column_letter}{row_number}"] = CellRichText(
        text[:quote_start],
        TextBlock(InlineFont(b=True), text[quote_start:quote_end].replace("**", "")),
        text[quote_end:],
    )

# Adjust column widths
column_widths = {
    "id": 12,
    "channel_name": 15,
    "start": 20,
    "text": 80,
    "quote": 25,
    "claim": 25,
    "analysis": 30,
    "context": 30,
    "cards": 25,
}
for col, width in column_widths.items():
    worksheet.column_dimensions[
        get_column_letter(review_df.columns.tolist().index(col) + 1)
    ].width = width

writer.close()